In [45]:
import pandas as pd

In [48]:
def standard(x_train, x_test):
    stdsc = StandardScaler()
    # 訓練用のデータを標準化
    x_train_std = stdsc.fit_transform(x_train)
    # 訓練用データを基準にテストデータも標準化
    x_test_std = stdsc.transform(x_test)
    return x_train_std, x_test_std

In [46]:
def load_data():
    data = pd.read_pickle('data.pkl')
    nomination_onehot = pd.read_pickle('nomination_onehot.pkl')
    selected_performers_onehot = pd.read_pickle('selected_performers_onehot.pkl')
    selected_directors_onehot = pd.read_pickle('selected_directors_onehot.pkl')
    selected_studio_onehot = pd.read_pickle('selected_studio_onehot.pkl')
    selected_scriptwriter_onehot = pd.read_pickle('selected_scriptwriter_onehot.pkl')
    review_dataframe = pd.read_pickle('review_dataframe.pkl')
    
    # selected_directors_onehotとselected_scriptwriter_onehotの重複した人
    duplicate_scriptwriter = set(selected_directors_onehot.columns) & set(selected_scriptwriter_onehot.columns)
    selected_scriptwriter_onehot = selected_scriptwriter_onehot.drop(duplicate_scriptwriter, axis=1)
          
    df = pd.concat(
        [
            nomination_onehot, 
            selected_performers_onehot,
            selected_directors_onehot,
            selected_studio_onehot,
            selected_scriptwriter_onehot,
            data["screen_time"],
        ],
        axis=1
    )
    # 共線性の高いカラムを除く
    drop_clm = ['吉田一夫']
    df = df.drop(drop_clm, axis=1)
    
    # 取得できなかった上映時間(screen_time == -1)を平均で埋める
    df[df["screen_time"] == -1] = df.mean().screen_time

    # 正規化する
    df = (df-df.mean()) / df.std()
        
    # データセットとして扱うのに必要なyear, prizeのフラグを付与する
    df = pd.concat(
        [df, data["year"], data["prize"]], axis=1
    )
    
    return df

In [47]:
# save pandas.DataFrame as pickle
df = load_data()
df.to_pickle("unificated_data_set.pkl")